In [1]:
import json
import os

import pandas as pd
import toml

# ../src/manage.py dumpdata > dump.json

In [2]:
with open("dump_final.json", "r") as f:
    dump = json.load(f)

ticket_objects = [record for record in dump if record["model"] == "tickets.jiraticket"]
for ticket in ticket_objects:
    ticket["fields"]["jira_ticket"] = ticket["pk"]
tickets = pd.DataFrame(ticket["fields"] for ticket in ticket_objects)[["jira_ticket", "assignee", "reporter"]]
tickets.head()

,jira_ticket,assignee,reporter
0,DS-1002,mgorlick@mozilla.com,mloo@mozilla.com
1,DS-1005,msamuel@mozilla.com,sescalante@mozilla.com
2,DS-1010,msamuel@mozilla.com,sescalante@mozilla.com
3,DS-1011,ascholtz@mozilla.com,sescalante@mozilla.com
4,DS-1017,gkaberere@mozilla.com,gkaberere@mozilla.com


In [3]:
annotation_objects = [record for record in dump if record["model"] == "tickets.annotation"]
annotations = pd.DataFrame(annotation["fields"] for annotation in annotation_objects)
annotations.head()

,jira_ticket,abstract,deliverable,no_deliverable,product,topic,tags,artifact,title,completed_date
0,DS-1011,,,True,,,,,Jetstream: Error reporting,2020-10-26
1,DS-1005,,,True,,,,,Automated experiment analysis: Info Design Pro...,2020-09-02
2,DS-1010,,,True,,,,,automated experiment analysis - determining wh...,2020-09-22
3,DS-325,The performance effects of enabling WebRender ...,https://mozilla.report/post/projects/2020-02-f...,False,fenix,,webrender,experiment,WebRender vs. Non-WebRender Experiment for Gec...,2020-06-08
4,DS-146,We assessed the effect of our DoH deployment s...,https://mozilla-private.report/doh-engagement/...,False,desktop,platform,DoH,experiment,DNS over HTTPS engagement study and rollout,2020-11-06


In [4]:
annotated = tickets.merge(annotations, on="jira_ticket", how="inner").query("not no_deliverable")
annotated["complete"] = (annotated["deliverable"] != '') & (annotated["completed_date"].astype(bool))

In [5]:
annotated.head()

,jira_ticket,assignee,reporter,abstract,deliverable,no_deliverable,product,topic,tags,artifact,title,completed_date,complete
7,DS-1029,tbrooks@mozilla.com,mbanner@mozilla.com,This experiment tested an infrastructure chang...,https://sql.telemetry.mozilla.org/queries/75912,False,desktop,security,,experiment,Please advise Amazon Search Replacement study,2020-10-28,True
14,DS-1045,flawrence@mozilla.com,flawrence@mozilla.com,A population of users drawn from MAU has a dis...,https://docs.google.com/document/d/13myV9B07GR...,False,desktop,,KPIs,investigation,Investigate whether MAU or DAU are fairer repr...,2020-09-08,True
15,DS-1047,gkaberere@mozilla.com,gkaberere@mozilla.com,Replaced the weekly reporting databricks noteb...,https://datastudio.google.com/reporting/5f5f7d...,False,other,,"weekly reporting, dashboards",dashboard,Create KPI Dashboard to replace Weekly KPI Rep...,2020-09-28,True
17,DS-1053,cdowhygelund@mozilla.com,vtay@mozilla.com,Two potential interventions are being proposed...,https://docs.google.com/document/d/1Rsf5pkgFnK...,False,desktop,,"user behavior, opportunity sizing",investigation,Opportunity Size 2020:08 Fx/Win10 Download,2020-08-31,True
18,DS-1058,gkaberere@mozilla.com,athomas@mozilla.com,Ask to compare iPad vs iPhone performance for ...,https://docs.google.com/document/d/1ATq159uYRh...,False,fennec_ios,,mobile,investigation,[iOS] iPad Opportunity Investigation,2020-10-28,True


In [6]:
PRODUCT_CHOICES = dict([
    ("desktop", "Firefox desktop"),
    ("fenix", "Firefox for Android (Fenix)"),
    ("fennec", "Firefox for Android (Fennec)"),
    ("fennec_ios", "Firefox for iOS"),
    ("other", "Other"),
    ("web", "Web properties"),
    ("many", "Many products"),
    ("", "")
])

TOPICS = dict((
    ("accounts", "Accounts"),
    ("activity_stream", "Activity Stream"),
    ("addons", "Add-ons"),
    ("bookmarks", "Bookmarks"),
    ("devtools", "Devtools"),
    ("download", "Download page"),
    ("enterprise", "Enterprise"),
    ("geo", "Geolocation"),
    ("heartbeat", "Heartbeat"),
    ("nav", "Navigation"),
    ("newtab", "New tab"),
    ("marketing", "Marketing campaign"),
    ("onboarding", "Onboarding"),
    ("pwmgr", "Password manager"),
    ("payments", "Payments"),
    ("percperf", "Perceived performance"),
    ("perf", "Performance"),
    ("platform", "Platform"),
    ("pocket", "Pocket"),
    ("privacy", "Privacy"),
    ("search", "Search"),
    ("security", "Security"),
    ("services", "Services"),
    ("tp", "Tracking Protection"),
    ("vpn", "VPN"),
    ("", "")
))

ARTIFACTS = dict((
    ("dashboard", "Dashboard"),
    ("experiment", "Experiment"),
    ("investigation", "Investigation report"),
    ("survey", "Survey"),
    ("other", "Other"),
    ("", "")
))

AUTHORS = {
    "": " <>",
    "bmiroglio@mozilla.com": "Ben Miroglio <bmiroglio@mozilla.com>",
    "cdowhygelund@mozilla.com": "Corey Dow-Hygelund <cdowhygelund@mozilla.com>",
    "dxu@mozilla.com": "Danica Xu <dxu@mozilla.com>",
    "echo@mozilla.com": "Edward Cho <echo@mozilla.com>",
    "ethompson@mozilla.com": "Emily Thompson <ethompson@mozilla.com>",
    "flawrence@mozilla.com": "Felix Lawrence <flawrence@mozilla.com>",
    "gkaberere@mozilla.com": "George Kaberere <gkaberere@mozilla.com>",
    "jhollek@mozilla.com": "Julie Hollek <jhollek@mozilla.com>",
    "jmccrosky@mozilla.com": "Jesse McCrosky <jmccrosky@mozilla.com>",
    "ksiegler@mozilla.com": "Kimmy Siegler <ksiegler@mozilla.com>",
    "loines@mozilla.com": "Leif Oines <loines@mozilla.com>",
    "mgorlick@mozilla.com": "Marissa Gorlick <mgorlick@mozilla.com>",
    "mgrant@mozilla.com": "Marshall Grant <mgrant@mozilla.com>",
    "mmccorquodale@mozilla.com": "Megan McCorquodale <mmccorquodale@mozilla.com>",
    "mreid@mozilla.com": "Mark Reid <mreid@mozilla.com>",
    "msamuel@mozilla.com": "Marina Samuel <msamuel@mozilla.com>",
    "rharter@mozilla.com": "Ryan Harter <rharter@mozilla.com>",
    "sghose@mozilla.com": "Saptarshi Ghose <sghose@mozilla.com>",
    "sguha@mozilla.com": "Saptarshi Guha <sguha@mozilla.com>",
    "shong@mozilla.com": "Su-Young Hong <shong@mozilla.com>",
    "tbrooks@mozilla.com": "Teon Brooks <tbrooks@mozilla.com>",
    "tismith@mozilla.com": "Tim D. Smith <tdsmith@mozilla.com>",
    "wbeard@mozilla.com": "Chris Beard <wbeard@mozilla.com>",
    "xluo@mozilla.com": "Xuan Luo <xluo@mozilla.com>",
}

column_map = {
    "jira_ticket": "Jira ticket",
    "Edit": "Edit",
    "completed_date": "Date",
    "title": "Title",
    "deliverable": "Link",
    "abstract": "Summary",
    "assignee": "Author",
    "reporter": "Reporter",
    "artifact": "Artifact",
    "product": "Product",
    "topic": "Topics",
    "tags": "Tags",
}

In [7]:
spreadsheet = annotated.rename(columns=column_map)
spreadsheet["Topics"] = spreadsheet["Topics"].str.split(",").map(lambda x: (TOPICS[t] for t in x)).str.join(", ")
spreadsheet["Artifact"] = spreadsheet["Artifact"].map(lambda x: ARTIFACTS[x])
spreadsheet["Product"] = spreadsheet["Product"].map(lambda x: PRODUCT_CHOICES[x])
spreadsheet["Author"] = spreadsheet["Author"].map(lambda x: AUTHORS[x])
spreadsheet["Edit"] = '=HYPERLINK("https://deliverables.wiki/tickets/' + spreadsheet["Jira ticket"] + '", "Edit")'
spreadsheet["Link"] = '=HYPERLINK("' + spreadsheet["Link"] + '", "Link")'
complete = spreadsheet.loc[spreadsheet.complete, column_map.values()]

In [8]:
complete.head()

,Jira ticket,Edit,Date,Title,Link,Summary,Author,Reporter,Artifact,Product,Topics,Tags
7,DS-1029,"=HYPERLINK(""https://deliverables.wiki/tickets/...",2020-10-28,Please advise Amazon Search Replacement study,"=HYPERLINK(""https://sql.telemetry.mozilla.org/...",This experiment tested an infrastructure chang...,Teon Brooks <tbrooks@mozilla.com>,mbanner@mozilla.com,Experiment,Firefox desktop,Security,
14,DS-1045,"=HYPERLINK(""https://deliverables.wiki/tickets/...",2020-09-08,Investigate whether MAU or DAU are fairer repr...,"=HYPERLINK(""https://docs.google.com/document/d...",A population of users drawn from MAU has a dis...,Felix Lawrence <flawrence@mozilla.com>,flawrence@mozilla.com,Investigation report,Firefox desktop,,KPIs
15,DS-1047,"=HYPERLINK(""https://deliverables.wiki/tickets/...",2020-09-28,Create KPI Dashboard to replace Weekly KPI Rep...,"=HYPERLINK(""https://datastudio.google.com/repo...",Replaced the weekly reporting databricks noteb...,George Kaberere <gkaberere@mozilla.com>,gkaberere@mozilla.com,Dashboard,Other,,"weekly reporting, dashboards"
17,DS-1053,"=HYPERLINK(""https://deliverables.wiki/tickets/...",2020-08-31,Opportunity Size 2020:08 Fx/Win10 Download,"=HYPERLINK(""https://docs.google.com/document/d...",Two potential interventions are being proposed...,Corey Dow-Hygelund <cdowhygelund@mozilla.com>,vtay@mozilla.com,Investigation report,Firefox desktop,,"user behavior, opportunity sizing"
18,DS-1058,"=HYPERLINK(""https://deliverables.wiki/tickets/...",2020-10-28,[iOS] iPad Opportunity Investigation,"=HYPERLINK(""https://docs.google.com/document/d...",Ask to compare iPad vs iPhone performance for ...,George Kaberere <gkaberere@mozilla.com>,athomas@mozilla.com,Investigation report,Firefox for iOS,,mobile


In [9]:
complete.to_csv("spreadsheet.csv", index=False)

In [10]:
incomplete = spreadsheet.loc[~spreadsheet.complete, column_map.values()]
incomplete.to_csv("incomplete.csv", index=False)

In [15]:
docere_map = {
    "title": "title",
    "completed_date": "publish_date",
    "assignee": "author",
    "deliverable": "link",
    "abstract": "abstract",
    "topic": "areas",
    "product": "products",
    "tags": "tags",
    "artifact": "artifacts",
}
docere = annotated.rename(columns=docere_map).query("complete")
docere["areas"] = docere["areas"].str.split(",").map(lambda x: [TOPICS[t] for t in x])
docere["artifacts"] = docere["artifacts"].map(lambda x: ARTIFACTS[x])
docere["products"] = docere["products"].map(lambda x: [PRODUCT_CHOICES[x]])
docere["author"] = docere["author"].map(lambda x: AUTHORS[x])
docere["tags"] = docere["tags"].str.split(",").map(lambda x: [t.strip() for t in x if t.strip()])
docere = docere.set_index("jira_ticket")[docere_map.values()]

In [16]:
docere.head()

,title,publish_date,author,link,abstract,areas,products,tags,artifacts
jira_ticket,,,,,,,,,
DS-1029,Please advise Amazon Search Replacement study,2020-10-28,Teon Brooks <tbrooks@mozilla.com>,https://sql.telemetry.mozilla.org/queries/75912,This experiment tested an infrastructure chang...,[Security],[Firefox desktop],[],Experiment
DS-1045,Investigate whether MAU or DAU are fairer repr...,2020-09-08,Felix Lawrence <flawrence@mozilla.com>,https://docs.google.com/document/d/13myV9B07GR...,A population of users drawn from MAU has a dis...,[],[Firefox desktop],[KPIs],Investigation report
DS-1047,Create KPI Dashboard to replace Weekly KPI Rep...,2020-09-28,George Kaberere <gkaberere@mozilla.com>,https://datastudio.google.com/reporting/5f5f7d...,Replaced the weekly reporting databricks noteb...,[],[Other],"[weekly reporting, dashboards]",Dashboard
DS-1053,Opportunity Size 2020:08 Fx/Win10 Download,2020-08-31,Corey Dow-Hygelund <cdowhygelund@mozilla.com>,https://docs.google.com/document/d/1Rsf5pkgFnK...,Two potential interventions are being proposed...,[],[Firefox desktop],"[user behavior, opportunity sizing]",Investigation report
DS-1058,[iOS] iPad Opportunity Investigation,2020-10-28,George Kaberere <gkaberere@mozilla.com>,https://docs.google.com/document/d/1ATq159uYRh...,Ask to compare iPad vs iPhone performance for ...,[],[Firefox for iOS],[mobile],Investigation report


In [17]:
os.makedirs("kr/toml", exist_ok=True)
for key, value in docere.to_dict(orient="index").items():
    with open(os.path.join("kr/toml", key + ".toml"), "w") as f:
        toml.dump(value, f)